In [30]:
from ipywidgets import widgets
import networkx as nx
from IPython import display as dis
import random
import pylab
from matplotlib.pyplot import pause
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

n_nodes = 5
n_edges = 10
sec_pause = 3

g = nx.Graph()
nodes = {}
for i in range(n_nodes):
    g.add_node(i)
    nodes[i] = set()

edges_count = 0
while edges_count < n_edges:
    edge = (random.randint(0, n_nodes-1), random.randint(0, n_nodes-1))
    if edge[0] != edge[1]:
        if edge[1] not in nodes[edge[0]]:
            nodes[edge[0]].add(edge[1])
            nodes[edge[1]].add(edge[0])
            edges_count += 1
            g.add_edges_from([edge]) 
    
    
pos = nx.shell_layout(g)

color_map = ['k' for i in range(n_nodes)]

def node(step,node, color, color_map, sec):
    plt.figure(3,figsize=(10,10))
    if node is not None:
        plt.title('Iteration {}. Try node {} to color \"{}\"'.format(step, node, color))
    else:
        plt.title('Iteration {}.'.format(step))
    nx.draw(g, pos=pos, with_labels=True, node_color = color_map, node_size=500, width=1, font_color='w', font_size=16)
    plt.show()
    plt.pause(sec)
    dis.clear_output(wait=True)
    
        
class BackTrack:
    
    def __init__(self, assigment, nodes):
        self.counter = 0
        self.assigment = assigment
        self.nodes = nodes 
        self.colors = ['r', 'b', 'g']
        node(self.counter,None,None, self.assigment, sec_pause/4)
    
    def backTracking(self):
        
        
        if int(self.assigment.count('k')) == 0:
            return self.assigment
        to_assign = self.returnUnassigment()
        for color in self.colors:
            self.counter += 1
            node(self.counter,to_assign,color, self.assigment, sec_pause)
            borderColor = set()
            for i in self.nodes[to_assign]:
                borderColor.add(self.assigment[i])
            if color not in borderColor:
                self.assigment[to_assign] = color
                node("SET",None, None, self.assigment, sec_pause/4)
                color_map = self.backTracking()
                if color_map is not None:
                    return self.assigment
            node("CANT -> NEXT COLOR",None, None, self.assigment, sec_pause/4)
        node("CANT SET -> GO BACK",None, None, self.assigment, sec_pause/4)
        self.assigment[to_assign] = 'k'
        return None
    
    def returnUnassigment(self):
        for i, j in enumerate(self.assigment):
            if j == 'k':
                return i
    
    def getCnt(self):
        return self.counter

bac = BackTrack(color_map, nodes)
color_map = bac.backTracking()
if color_map is None:
    print("No solution, number of set colors: {}".format(bac.getCnt()))
else: 
    print(color_map)
    node("Finally find the sollution, number of set colors: {}".format(bac.getCnt()),None, None, color_map, 10)
    
    


No solution, number of set colors: 48
